# Optimal transport in $\mathbb{R}^2$

Let $X \sim N(0_{N}, \Sigma_{X})$ and $Y \sim N(0_{N}, \Sigma_{Y})$ be two bivariate gaussian distributions.

In [3]:
import Random
import LinearAlgebra
import CovarianceEstimation
using Distributions
using Plots
using StatsPlots
seed = 1234
gen = Random.Xoshiro(seed);

In [4]:
# Covariance matrix of X
ΣX = [0.5 0.; 0. 0.5];
# Covariance matrix of Y
ΣY = [2. 0.; 0. 2.];

In [5]:
# Distributions of X ~ N(0, I_n) and Y ~ N(0, sqrt(2) * I_n)
distμ = MvNormal(ΣX);
distν = MvNormal(ΣY);

In [6]:
N = 100;
x = rand(distμ, N);
y = rand(distν, N);

In [7]:
# pdf of X and Y
f = Distributions.pdf(distμ, x);
g = Distributions.pdf(distν, y);

**Density function of X**

In [ ]:
surface(x[1,:], x[2, :], f, c=:PuBu, colorbar = :none)

**Density function of Y**

In [ ]:
surface(y[1,:], y[2, :], g, c=:OrRd, colorbar=:none)

**Density function of X ($f$) and density function of Y ($g$)**

In [ ]:
surface(y[1,:], y[2, :], g, c=:OrRd, colorbar = :none)
surface!(x[1,:], x[2, :], f, c=:PuBu, alpha = 0.4, colorbar = :none)

**Loss function**

$$c_{T}(x, y) = \frac{1}{2} \|x - y\|^2$$

**Optimal transportation map of X to Y**

If $X \sim N(0, \Sigma_{X})$ and $Y \sim N(0, \Sigma_{Y})$, and the cost function is $c_{T}(x,y) = \frac{1}{2} \|Ax-y\|^2$ where $A$ is invertible, the optimal transportation map $T$ of $X$ to $Y$ has a closed form solution given by

$$
T : X -> Y \\
T(x) = (A^{T})^{-1} \Sigma_{X}^{-1/2}(\Sigma_{X}^{1/2} A^{T} \Sigma_{Y} A \Sigma_{X}^{1/2})^{1/2} \Sigma_{X}^{-1/2} x
$$

If $A = I_{n}$ then

$$
\begin{align}
T(x) &= \Sigma_{X}^{-1/2}(\Sigma_{X}^{1/2} \Sigma_{Y} \Sigma_{X}^{1/2})^{1/2} \Sigma_{X}^{-1/2} x \\
    &= \Sigma_{X}^{-1}(\Sigma_{X} \Sigma_{Y})^{1/2} x
\end{align}
$$

In [ ]:
function optimal_fn(x, μ, ν)
    ΣX = cov(μ)
    ΣY = cov(ν)
    v = inv(ΣX) * ((ΣX * ΣY) ^ (1/2))
    n = size(x, 2)
    tx = zeros(2, n)
    for i in 1:n
        tx[:, i] = v * x[:, i]
    end
    return tx
end

In [ ]:
tx = optimal_fn(x, distμ, distν);

**Visual checking**

To plot the distribution of the map of $X$, we assume that $T(X) \sim N(0_{N}, \Sigma_{\gamma})$
where $\Sigma_{\gamma}$ is the estimated covariance matrix of $T(X)$.

In [ ]:
Σγ = CovarianceEstimation.cov(CovarianceEstimation.SimpleCovariance(), transpose(tx))

In [ ]:
distγ = MvNormal(Σγ)

In [ ]:
h = Distributions.pdf(distγ, tx);

In [ ]:
surface(y[1,:], y[2, :], g, c=:OrRd, colorbar = :none)
surface!(tx[1,:], tx[2, :], h, c=:PuBu, alpha = 0.4, colorbar = :none)

The distributions of $T(X)$ and $Y$ have similar shapes. This can be improved by sampling more points but it requires a lot more computing ressources.